In [43]:
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("OPENROUTER_API_KEY")

def call_llm(prompt, model="mistralai/mistral-7b-instruct"):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.3
    }

    response = requests.post(url, headers=headers, json=payload, timeout=15)
    data = response.json()

    # Debug: print full response if 'choices' is missing
    if "choices" not in data:
        print("⚠️ API Error:", data)
        return json.dumps({"predicted_stars": None, "explanation": "API error or invalid response"})

    return data["choices"][0]["message"]["content"]


In [44]:
import pandas as pd

# Load the dataset
df = pd.read_csv("yelp.csv")  # adjust path if needed

# Keep only review text and star rating
df = df[["text", "stars"]].dropna()
df = df.rename(columns={"text": "review_text"})

# Sample ~200 reviews (stratified across ratings if possible)
sample = (
    df.groupby("stars", group_keys=False)
      .apply(lambda x: x.sample(min(len(x), 40), random_state=42))
      .reset_index(drop=True)
)

sample.head()


/var/folders/r2/b7kkz39s5ml7bc3fnjm727_00000gn/T/ipykernel_2389/1999904717.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(min(len(x), 40), random_state=42))


,review_text,stars
0,An upscale mexican restaurant in the area seem...,1
1,So i thought with all the controversy surround...,1
2,The only reason 1 star can be given is the fla...,1
3,I am leaving this store angry and can't resist...,1
4,We came here for dinner for me and reverse hap...,1


In [60]:
def prompt_direct(review_text):
    return f"""You are a helpful assistant that predicts Yelp star ratings.

Review: {review_text}

Respond ONLY with a valid JSON object in this format:
{{
  "predicted_stars": 1 to 5,
  "explanation": "Your reasoning here"
}}

Do not include any other text or formatting."""



def prompt_fewshot(review_text):
    return f"""You are a Yelp rating predictor. Use examples to map sentiment to stars.
Return strictly JSON with keys "predicted_stars" and "explanation".

{FEW_SHOT_EXAMPLES}

Now classify:
Review: "{review_text}"
Output JSON only:"""


FEW_SHOT_EXAMPLES = """
Review: "Amazing service and delicious food!"
Output: {"predicted_stars": 5, "explanation": "Positive sentiment: great service and food"}

Review: "Terrible experience. Will not return."
Output: {"predicted_stars": 1, "explanation": "Negative sentiment: bad experience"}

Review: "Decent food, but a bit pricey."
Output: {"predicted_stars": 3, "explanation": "Mixed sentiment: okay food, cost concern"}
"""

def prompt_fewshot(review_text):
    return f"""You are a Yelp rating predictor. Use examples to map sentiment to stars.
Return strictly JSON with keys "predicted_stars" and "explanation".

{FEW_SHOT_EXAMPLES}

Now classify:
Review: "{review_text}"
Output JSON only:"""


In [53]:
prompt = prompt_direct(sample.iloc[0]["review_text"])
response = call_llm(prompt, model="mistralai/mistral-7b-instruct")
print("🧠 Raw response:\n", response)


🧠 Raw response:
  {
  "predicted_stars": 2,
  "explanation": "The review highlights several negative aspects including poor service, long wait times, and unappetizing food. The waiter was described as snobby and inattentive, and the food was not well-received. The overall experience was disappointing, warranting a 2-star rating."
}


In [54]:
def parse_json_safe(text: str):
    text = text.strip()
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        # Try to extract JSON substring if extra text is around it
        start = text.find("{")
        end = text.rfind("}")
        if start != -1 and end != -1 and end > start:
            candidate = text[start:end+1]
            try:
                return json.loads(candidate)
            except:
                return None
        return None


In [56]:
import time

results = []
start = time.time()

for i, row in sample.head(200).iterrows():
    print(f"\n🔹 Review {i+1}")
    prompt = prompt_direct(row["review_text"])

    for attempt in range(3):  # try up to 3 times
        try:
            response = call_llm(prompt, model="mistralai/mistral-7b-instruct")
            parsed = parse_json_safe(response)
            break  # exit retry loop if successful
        except Exception as e:
            print(f"⚠️ Attempt {attempt+1} failed: {e}")
            time.sleep(2)  # wait before retrying
            parsed = None

    if parsed and "predicted_stars" in parsed and "explanation" in parsed:
        results.append({
            "actual": row["stars"],
            "predicted": parsed["predicted_stars"],
            "explanation": parsed["explanation"]
        })
    else:
        results.append({
            "actual": row["stars"],
            "predicted": None,
            "explanation": "API error or invalid JSON"
        })

end = time.time()
print("⏱ Total time:", round(end - start, 2), "seconds")



🔹 Review 1

🔹 Review 2

🔹 Review 3

🔹 Review 4

🔹 Review 5

🔹 Review 6

🔹 Review 7

🔹 Review 8

🔹 Review 9

🔹 Review 10

🔹 Review 11

🔹 Review 12

🔹 Review 13
⚠️ Attempt 1 failed: HTTPSConnectionPool(host='openrouter.ai', port=443): Read timed out. (read timeout=15)

🔹 Review 14

🔹 Review 15

🔹 Review 16

🔹 Review 17

🔹 Review 18

🔹 Review 19

🔹 Review 20

🔹 Review 21

🔹 Review 22

🔹 Review 23
⚠️ Attempt 1 failed: HTTPSConnectionPool(host='openrouter.ai', port=443): Read timed out. (read timeout=15)

🔹 Review 24

🔹 Review 25

🔹 Review 26

🔹 Review 27

🔹 Review 28

🔹 Review 29

🔹 Review 30

🔹 Review 31

🔹 Review 32

🔹 Review 33

🔹 Review 34

🔹 Review 35
⚠️ Attempt 1 failed: HTTPSConnectionPool(host='openrouter.ai', port=443): Read timed out. (read timeout=15)
⚠️ Attempt 2 failed: HTTPSConnectionPool(host='openrouter.ai', port=443): Read timed out. (read timeout=15)

🔹 Review 36

🔹 Review 37

🔹 Review 38
⚠️ Attempt 1 failed: HTTPSConnectionPool(host='openrouter.ai', port=443): Read time

In [57]:
df = pd.DataFrame(results)
print("✅ Reviews processed:", len(df))
print("🔍 Valid predictions:", df["predicted"].notna().sum())


✅ Reviews processed: 200
🔍 Valid predictions: 195


In [58]:
remaining = sample.iloc[len(df):]


In [59]:
valid = df[df["predicted"].notna()]
accuracy = (valid["actual"] == valid["predicted"]).mean()
print("🎯 Accuracy:", round(accuracy, 4))


🎯 Accuracy: 0.5641


In [61]:
results_reasoning = []
import time

start = time.time()
for i, row in sample.head(200).iterrows():
    print(f"\n🔹 Reasoning Review {i+1}")
    prompt = prompt_reasoning(row["review_text"])
    response = call_llm(prompt, model="mistralai/mistral-7b-instruct")
    parsed = parse_json_safe(response)

    if parsed and "predicted_stars" in parsed and "explanation" in parsed:
        results_reasoning.append({
            "actual": row["stars"],
            "predicted": parsed["predicted_stars"],
            "explanation": parsed["explanation"]
        })
    else:
        results_reasoning.append({
            "actual": row["stars"],
            "predicted": None,
            "explanation": "Invalid JSON"
        })

end = time.time()
print("⏱ Total time:", round(end - start, 2), "seconds")

df_reasoning = pd.DataFrame(results_reasoning)
print("🟢 Reviews processed:", len(df_reasoning))
print("🟢 Valid predictions:", df_reasoning["predicted"].notna().sum())

valid_reasoning = df_reasoning[df_reasoning["predicted"].notna()]
accuracy_reasoning = (valid_reasoning["actual"] == valid_reasoning["predicted"]).mean()
print("📉 Accuracy (Reasoning):", round(accuracy_reasoning, 4))



🔹 Reasoning Review 1

🔹 Reasoning Review 2

🔹 Reasoning Review 3

🔹 Reasoning Review 4

🔹 Reasoning Review 5

🔹 Reasoning Review 6

🔹 Reasoning Review 7

🔹 Reasoning Review 8

🔹 Reasoning Review 9

🔹 Reasoning Review 10

🔹 Reasoning Review 11

🔹 Reasoning Review 12

🔹 Reasoning Review 13

🔹 Reasoning Review 14

🔹 Reasoning Review 15

🔹 Reasoning Review 16

🔹 Reasoning Review 17

🔹 Reasoning Review 18

🔹 Reasoning Review 19

🔹 Reasoning Review 20

🔹 Reasoning Review 21

🔹 Reasoning Review 22

🔹 Reasoning Review 23

🔹 Reasoning Review 24

🔹 Reasoning Review 25

🔹 Reasoning Review 26

🔹 Reasoning Review 27

🔹 Reasoning Review 28

🔹 Reasoning Review 29

🔹 Reasoning Review 30

🔹 Reasoning Review 31

🔹 Reasoning Review 32

🔹 Reasoning Review 33

🔹 Reasoning Review 34

🔹 Reasoning Review 35

🔹 Reasoning Review 36

🔹 Reasoning Review 37

🔹 Reasoning Review 38

🔹 Reasoning Review 39

🔹 Reasoning Review 40

🔹 Reasoning Review 41

🔹 Reasoning Review 42

🔹 Reasoning Review 43

🔹 Reasoning Review 

In [62]:
results_fewshot = []
import time

start = time.time()
for i, row in sample.head(200).iterrows():
    print(f"\n🔹 Few-shot Review {i+1}")
    prompt = prompt_fewshot(row["review_text"])
    response = call_llm(prompt, model="mistralai/mistral-7b-instruct")
    parsed = parse_json_safe(response)

    if parsed and "predicted_stars" in parsed and "explanation" in parsed:
        results_fewshot.append({
            "actual": row["stars"],
            "predicted": parsed["predicted_stars"],
            "explanation": parsed["explanation"]
        })
    else:
        results_fewshot.append({
            "actual": row["stars"],
            "predicted": None,
            "explanation": "Invalid JSON"
        })

end = time.time()
print("⏱ Total time:", round(end - start, 2), "seconds")

df_fewshot = pd.DataFrame(results_fewshot)
print("🟢 Reviews processed:", len(df_fewshot))
print("🟢 Valid predictions:", df_fewshot["predicted"].notna().sum())

valid_fewshot = df_fewshot[df_fewshot["predicted"].notna()]
accuracy_fewshot = (valid_fewshot["actual"] == valid_fewshot["predicted"]).mean()
print("📉 Accuracy (Few-shot):", round(accuracy_fewshot, 4))



🔹 Few-shot Review 1

🔹 Few-shot Review 2

🔹 Few-shot Review 3

🔹 Few-shot Review 4

🔹 Few-shot Review 5

🔹 Few-shot Review 6

🔹 Few-shot Review 7

🔹 Few-shot Review 8

🔹 Few-shot Review 9

🔹 Few-shot Review 10

🔹 Few-shot Review 11

🔹 Few-shot Review 12

🔹 Few-shot Review 13

🔹 Few-shot Review 14

🔹 Few-shot Review 15

🔹 Few-shot Review 16

🔹 Few-shot Review 17

🔹 Few-shot Review 18

🔹 Few-shot Review 19

🔹 Few-shot Review 20

🔹 Few-shot Review 21

🔹 Few-shot Review 22

🔹 Few-shot Review 23

🔹 Few-shot Review 24

🔹 Few-shot Review 25

🔹 Few-shot Review 26

🔹 Few-shot Review 27

🔹 Few-shot Review 28

🔹 Few-shot Review 29

🔹 Few-shot Review 30

🔹 Few-shot Review 31

🔹 Few-shot Review 32

🔹 Few-shot Review 33

🔹 Few-shot Review 34

🔹 Few-shot Review 35

🔹 Few-shot Review 36

🔹 Few-shot Review 37

🔹 Few-shot Review 38

🔹 Few-shot Review 39

🔹 Few-shot Review 40

🔹 Few-shot Review 41

🔹 Few-shot Review 42

🔹 Few-shot Review 43

🔹 Few-shot Review 44

🔹 Few-shot Review 45

🔹 Few-shot Review 

In [63]:
comparison = pd.DataFrame([
    {"Prompt": "Direct", "Accuracy": round(accuracy, 4), "JSON Validity": df["predicted"].notna().mean()},
    {"Prompt": "Reasoning", "Accuracy": round(accuracy_reasoning, 4), "JSON Validity": df_reasoning["predicted"].notna().mean()},
    {"Prompt": "Few-shot", "Accuracy": round(accuracy_fewshot, 4), "JSON Validity": df_fewshot["predicted"].notna().mean()},
])
comparison


,Prompt,Accuracy,JSON Validity
0,Direct,0.5641,0.975
1,Reasoning,0.4872,0.195
2,Few-shot,0.6300,1.000
